In [2]:
import mysql.connector
import pandas as pd
import glob
import numpy as np
from functools import reduce

In [3]:


# declare a connection to the MySQL server 
# the hostname (or IP) of the server machine should be passed

db = mysql.connector.connect(
  user="root",
  password="R13251618a", 
  host="127.0.0.1",)


In [4]:
# Printing the connection object
print(db)


In [5]:
# creating the db cursor object
dbc = db.cursor()

In [8]:
#dbc.execute("SET GLOBAL local_infile = 'ON';")

In [5]:
# Showing all the databases

dbc.execute(
"SHOW DATABASES"
)

# loop over the container to fetch the results
for _ in dbc:
    print(_)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('World_Cup',)


In [6]:
# Creating the WorldCup database if it does not exist

creation_instruction = "CREATE DATABASE IF NOT EXISTS World_Cup;"

dbc.execute(
creation_instruction
)

db.commit()

In [7]:
# Checking if the database has been created

dbc.execute(
"SHOW DATABASES"
)
for _ in dbc:
    print(_)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('World_Cup',)


In [6]:
database_to_use = "World_cup"

dbc.execute(f"USE {database_to_use};")

In [71]:
#dbc.execute("ALTER TABLE Races ADD COLUMN Sex CHAR(1) AFTER Date;")


In [72]:
#db.commit()

In [73]:
dbc.execute("DESCRIBE Races;")
res = dbc.fetchall()
pd.DataFrame(res)

,0,1,2,3,4,5
0,RaceID,int,NO,PRI,None,auto_increment
1,Season,char(5),NO,,None,
2,Format,varchar(20),NO,,None,
3,Location,varchar(20),NO,,None,
4,Nation,char(3),NO,,None,
5,Date,date,NO,,None,
6,Sex,char(1),YES,,None,
7,Total_km,smallint,NO,,None,
8,Wind_30_before,varchar(20),YES,,None,
9,Wind_at_start,varchar(20),YES,,None,


In [8]:
dbc.execute("SELECT * FROM Races;")
res = dbc.fetchall()
pd.DataFrame(res)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,1,22-23,Individual,Kontiolahti,FIN,2022-11-29,M,20,E 1.0,E 1.2,...,-3.2,-3.4,90.0,90.0,90.0,90.0,20113,37,34,635
1,2,22-23,Individual,Kontiolahti,FIN,2022-11-30,F,15,E 1.4,E 1.9,...,-3.2,-3.2,91.0,91.0,91.0,91.0,15238,37,34,510


### Creation of the Athletes table

In [10]:
# Creation of the Athletes table


#Names_table = "CREATE TABLE IF NOT EXISTS Athletes (\
#        AthleteID INT NOT NULL UNIQUE AUTO_INCREMENT PRIMARY KEY,\
#        FirstName varchar(20) NOT NULL,\
#        LastName varchar(20) NOT NULL,\
#        Nationality char(3) NOT NULL\
#        );"

In [11]:


#dbc.execute(Names_table)
#db.commit()

In [108]:
# Adding the FullName column to the Athletes table


#add_column = """
#ALTER TABLE Athletes ADD UNIQUE (FullName);
#"""

In [109]:
# Committing the changes


#dbc.execute(add_column)
#db.commit()

In [10]:
# Checking the structure of the table

dbc.execute("DESCRIBE Athletes;")
res_ = dbc.fetchall()
display(res_)

[('AthleteID', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('FullName', 'varchar(40)', 'NO', 'UNI', None, ''),
 ('FirstName', 'varchar(20)', 'NO', '', None, ''),
 ('LastName', 'longtext', 'NO', '', None, ''),
 ('Nationality', 'char(3)', 'NO', '', None, '')]

### Filling the Athletes table

In [59]:
athletes_files = glob.glob("../../../BiathlonAzzurroWrapper/data/data_RAE/athletes/*")


In [60]:
for i, file in enumerate(athletes_files):
    if '_w' in file:
        sex = 'F'
    else:
        sex = 'M'
    if i == 0:
        final_df = pd.read_csv(file, sep="\t")
        final_df['sex'] = sex
    else:
        df = pd.read_csv(file, sep="\t", )
        df['sex'] = sex
        final_df = pd.concat([final_df, df])
    
    

final_df = final_df.reset_index(drop=True)
final_df = final_df[['Family\xa0Name', 'Given Name', 'Nation', 'sex']]
final_df.columns = ['Family Name', 'Given Name', 'Nation', 'sex']
final_df['Full Name'] = final_df['Given Name'] + " " + final_df['Family Name']
final_df['Given Name'][final_df['Full Name'] == "Alexandr Loginov"] = "Alexander"
final_df['Full Name'][final_df['Full Name'] == "Alexandr Loginov"] = "Alexander Loginov"
#print(len(final_df.groupby('Full Name').count().sort_values('Nation', ascending=False)))

group_by_df = pd.DataFrame(final_df.groupby(['Full Name', 'Nation', 'Family Name', 'Given Name', 'sex']).size(), columns = ['count'])
group_by_df = group_by_df.reset_index()
#group_by_df['final_nation'] = group_by_df.apply(lambda x: )
idx = group_by_df.groupby(['Full Name'])['count'].transform(max) == group_by_df['count']
final_athletes_df = group_by_df[idx]
final_athletes_df = final_athletes_df.drop_duplicates(subset=['Full Name'])
final_athletes_df = final_athletes_df.reset_index(drop=True)

#final_athletes_df.to_csv("../../../BiathlonAzzurroWrapper/data/data_RAE/athletes/final_athletes_sql.csv", index=False)


In [61]:
final_athletes_df = final_athletes_df.iloc[:, :-1]   
final_athletes_df                   

,Full Name,Nation,Family Name,Given Name,sex
0,Adam Kwak,POL,Kwak,Adam,M
1,Adam Runnalls,CAN,Runnalls,Adam,M
2,Adam Vaclavik,CZE,Vaclavik,Adam,M
3,Adele Walker,GBR,Walker,Adele,F
4,Adina Sotropa,ROM,Sotropa,Adina,F
...,...,...,...,...,...
2316,Zuzana Hasillova,SVK,Hasillova,Zuzana,F
2317,Zuzana Remenova,SVK,Remenova,Zuzana,F
2318,Zuzana Tryznova,CZE,Tryznova,Zuzana,F
2319,Zvetana Krasteva,BUL,Krasteva,Zvetana,F


In [51]:
#loading_table = "\
#    LOAD DATA LOCAL INFILE '../../../BiathlonAzzurroWrapper/data/data_RAE/athletes/final_athletes_sql.csv'\
#    INTO TABLE Athletes\
#    FIELDS TERMINATED BY ','\
#    IGNORE 1 ROWS\
#    (FirstName, LastName, Nationality);"

In [62]:

adding_athletes = list(map(lambda x: tuple(x) , np.array(final_athletes_df)))

#f'{list(map(lambda x: tuple(x), np.array(final_athletes_df)))}'.replace("'", "")
#adding_athletes_reduce = reduce(lambda x, y: x + y, adding_athletes, '')
#adding_athletes_reduce = adding_athletes_reduce[:-1]
#adding_athletes_reduce += ';'


In [9]:
# Checking the adding_athletes contents


#adding_athletes

In [67]:
# Filling the table


athletes_statement = """INSERT INTO Athletes (FullName, Nationality, LastName, FirstName, Sex) VALUES (%s, %s, %s, %s, %s);"""

val_ = adding_athletes

dbc.executemany(athletes_statement, val_)
db.commit()
#dbc.executemany(athletes_statement, np.array(final_athletes_df))

In [68]:
# Cecking if the Alexander Loginov problem has been solved


dbc.execute("SELECT * FROM Athletes WHERE FirstName = 'Alexandr' LIMIT 5;")
res_ = dbc.fetchall()
display(res_)

[(51, 'Alexandr Chervyakov', 'Alexandr', 'Chervyakov', 'KAZ', 'M'),
 (52, 'Alexandr Lysenko', 'Alexandr', 'Lysenko', 'UKR', 'M'),
 (53, 'Alexandr Mukhin', 'Alexandr', 'Mukhin', 'KAZ', 'M'),
 (54, 'Alexandr Popov', 'Alexandr', 'Popov', 'BLR', 'M'),
 (55, 'Alexandr Shesler', 'Alexandr', 'Shesler', 'KAZ', 'M')]

## Creating the Races Table

In [11]:
Races_table = "CREATE TABLE IF NOT EXISTS Races (\
        RaceID INT NOT NULL UNIQUE AUTO_INCREMENT PRIMARY KEY,\
        Season char(5) NOT NULL,\
        Format varchar(20) NOT NULL,\
        Location varchar(20) NOT NULL,\
        Nation char(3) NOT NULL,\
        Date date NOT NULL,\
        Wind_30_before varchar(20),\
        Wind_at_start varchar(20),\
        Wind_30_after varchar(20),\
        Wind_at_end varchar(20),\
        Snow_condition_30_before varchar(20),\
        Snow_condition_at_start varchar(20),\
        Snow_condition_30_after varchar(20),\
        Snow_condition_at_end varchar(20),\
        Temperature_snow_30_before varchar(20),\
        Temperature_snow_at_start varchar(20),\
        Temperature_snow_30_after varchar(20),\
        Temperature_snow_at_end varchar(20),\
        Temperature_air_30_before varchar(20),\
        Temperature_air_at_start varchar(20),\
        Temperature_air_30_after varchar(20),\
        Temperature_air_at_end varchar(20),\
        Humidity_30_before varchar(20),\
        Humidity_at_start varchar(20),\
        Humidity_30_after varchar(20),\
        Humidity_at_end varchar(20),\
        Total_course_length INT,\
        Height_difference INT,\
        Max_climb INT,\
        Total_climb INT);"

### Creating the Results table

In [90]:
Results_table = "CREATE TABLE IF NOT EXISTS Results (\
        RaceID INT NOT NULL,\
        FinalRank SMALLINT NOT NULL,\
        FullName varchar(30) NOT NULL,\
        FamilyName varchar(20) NOT NULL,\
        GivenName varchar(20) NOT NULL,\
        Nation char(3) NOT NULL,\
        TotalMistakes smallint NOT NULL,\
        Mistakes_1 smallint NOT NULL,\
        Mistakes_2 smallint NOT NULL,\
        Mistakes_3 smallint,\
        Mistakes_4 smallint,\
        TotalTime time,\
        BehindLeader time,\
        PRIMARY KEY (RaceID, FinalRank));"

In [91]:
dbc.execute(Results_table)
db.commit()

In [60]:
dbc.execute("DESCRIBE Results;")
res_ = dbc.fetchall()
res_


array(['RaceID', 'FinalRank', 'Bib', 'FullName', 'FamilyName',
       'GivenName', 'Nation', 'TotalMistakes', 'Mistakes_1', 'Mistakes_2',
       'Mistakes_3', 'Mistakes_4', 'TotalTime', 'BehindLeader'],
      dtype=object)

In [61]:
string_columns = reduce(lambda x, y: x + y + ", ", tmp, '')
string_columns = string_columns[:-2]
string_columns

'RaceID, FinalRank, Bib, FullName, FamilyName, GivenName, Nation, TotalMistakes, Mistakes_1, Mistakes_2, Mistakes_3, Mistakes_4, TotalTime, BehindLeader'

In [113]:
dbc.execute("DESCRIBE Results;")
res_ = dbc.fetchall()
res_

[('RaceID', 'int', 'NO', 'PRI', None, ''),
 ('FinalRank', 'smallint', 'NO', '', None, ''),
 ('Bib', 'smallint', 'NO', '', None, ''),
 ('FullName', 'varchar(30)', 'NO', 'PRI', None, ''),
 ('FamilyName', 'varchar(20)', 'NO', '', None, ''),
 ('GivenName', 'varchar(20)', 'NO', '', None, ''),
 ('Nation', 'char(3)', 'NO', '', None, ''),
 ('TotalMistakes', 'smallint', 'NO', '', None, ''),
 ('Mistakes_1', 'smallint', 'NO', '', None, ''),
 ('Mistakes_2', 'smallint', 'NO', '', None, ''),
 ('Mistakes_3', 'smallint', 'YES', '', None, ''),
 ('Mistakes_4', 'smallint', 'YES', '', None, ''),
 ('TotalTime', 'time(1)', 'YES', '', None, ''),
 ('BehindLeader', 'time(1)', 'YES', '', None, '')]

## Creating the loops table

In [12]:
create_loops = "CREATE TABLE IF NOT EXISTS Loops (\
    LoopID varchar(3) NOT NULL,\
    RaceID INT NOT NULL,\
    FinalRank SMALLINT NOT NULL,\
    Bib SMALLINT,\
    FullName varchar(30) NOT NULL,\
    FamilyName varchar(20) NOT NULL,\
    GivenName varchar(20) NOT NULL,\
    Nation char(3) NOT NULL,\
    CumulativeTime time,\
    SkiTime time,\
    LoopTime time,\
    CourseTime time,\
    RangeTime time,\
    ShootingTime time,\
    PenaltyTime time,\
    PRIMARY KEY (LoopID, RaceID));" 
    

In [52]:
dbc.execute("SELECT * FROM Loops\
    ORDER BY FinalRank;")
#dbc.execute("TRUNCATE Loops;")
res_ = dbc.fetchall()
pd.DataFrame(res_)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,1,1,28,Martin Ponsiluoma,Ponsiluoma,Martin,SWE,0 days 00:09:33,0 days 00:09:33,0 days 00:09:33,0 days 00:08:32,0 days 00:00:53,0 days 00:00:28,0 days 00:00:08
1,1,1,2,18,Niklas Hartweg,Hartweg,Niklas,SUI,0 days 00:09:47,0 days 00:09:47,0 days 00:09:47,0 days 00:08:47,0 days 00:00:51,0 days 00:00:25,0 days 00:00:09
2,1,1,3,74,David Zobel,Zobel,David,GER,0 days 00:09:46,0 days 00:09:46,0 days 00:09:46,0 days 00:08:42,0 days 00:00:55,0 days 00:00:31,0 days 00:00:09
3,1,1,4,33,Roman Rees,Rees,Roman,GER,0 days 00:09:42,0 days 00:09:42,0 days 00:09:42,0 days 00:08:40,0 days 00:00:52,0 days 00:00:28,0 days 00:00:09
4,1,1,5,11,Sebastian Samuelsson,Samuelsson,Sebastian,SWE,0 days 00:09:43,0 days 00:09:43,0 days 00:09:43,0 days 00:08:39,0 days 00:00:56,0 days 00:00:29,0 days 00:00:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1,1,93,69,Raido Raenkel,Raenkel,Raido,EST,0 days 00:10:04,0 days 00:10:04,0 days 00:10:04,0 days 00:08:43,0 days 00:01:12,0 days 00:00:46,0 days 00:00:09
93,1,1,94,76,Jan Gunka,Gunka,Jan,POL,0 days 00:12:27,0 days 00:10:27,0 days 00:12:27,0 days 00:09:16,0 days 00:01:02,0 days 00:00:36,0 days 00:02:09
94,1,1,95,65,Jin Nakajima,Nakajima,Jin,JPN,0 days 00:11:41,0 days 00:10:41,0 days 00:11:41,0 days 00:09:28,0 days 00:01:03,0 days 00:00:30,0 days 00:01:10
95,1,1,96,64,Sergey Sirik,Sirik,Sergey,KAZ,0 days 00:11:50,0 days 00:10:50,0 days 00:11:50,0 days 00:09:37,0 days 00:01:04,0 days 00:00:38,0 days 00:01:09


## Creating the shooting table

In [86]:
create_shooting = "CREATE TABLE IF NOT EXISTS Shooting (\
    RaceID INT NOT NULL PRIMARY KEY,\
    FinalRank SMALLINT NOT NULL,\
    Bib SMALLINT,\
    FullName varchar(30) NOT NULL,\
    FamilyName varchar(20) NOT NULL,\
    GivenName varchar(20) NOT NULL,\
    Nation char(3) NOT NULL,\
    Lane1 SMALLINT NOT NULL,\
    Shooting1 varchar(40) NOT NULL,\
    Time1 time NOT NULL,\
    Lane2 SMALLINT NOT NULL,\
    Shooting2 varchar(40) NOT NULL,\
    Time2 time NOT NULL,\
    Lane3 SMALLINT,\
    Shooting3 varchar(40),\
    Time3 time,\
    Lane4 SMALLINT,\
    Shooting4 varchar(40),\
    Time4 time);"

In [88]:
dbc.execute("DESCRIBE Shooting;")
res_ = dbc.fetchall()
pd.DataFrame(res_)


,0,1,2,3,4,5
0,RaceID,int,NO,PRI,None,
1,FinalRank,smallint,NO,,None,
2,Bib,smallint,YES,,None,
3,FullName,varchar(30),NO,,None,
4,FamilyName,varchar(20),NO,,None,
5,GivenName,varchar(20),NO,,None,
6,Nation,char(3),NO,,None,
7,Lane1,smallint,NO,,None,
8,Shooting1,varchar(40),NO,,None,
9,Time1,time,NO,,None,


In [36]:
dbc.execute("SELECT * FROM Shooting ORDER BY FinalRank LIMIT 5;")
res_ = dbc.fetchall()
pd.DataFrame(res_)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,1,28,Martin Ponsiluoma,Ponsiluoma,Martin,SWE,6,14.4s 3.4s 2.6s 2.5s 2.5s 54321,0 days 00:00:28,18,13.6s 2.5s 2.5s 2.4s 2.3s 04321,0 days 00:00:26,6,15.3s 2.3s 2.2s 2.4s 2.5s 54321,0 days 00:00:27,20,13.5s 2.1s 2.3s 6.1s 2.6s 54321,0 days 00:00:29
1,1,2,18,Niklas Hartweg,Hartweg,Niklas,SUI,9,13.1s 2.4s 2.2s 2.4s 2.1s 12345,0 days 00:00:25,19,13.2s 2.4s 2.6s 2.5s 2.6s 54321,0 days 00:00:26,9,16.4s 2.6s 2.3s 2.1s 2.1s 12345,0 days 00:00:28,17,11.6s 2.8s 2.3s 2.3s 2.3s 54321,0 days 00:00:24
2,1,3,74,David Zobel,Zobel,David,GER,3,16.6s 3.6s 2.5s 2.3s 2.8s 12345,0 days 00:00:31,18,16.4s 4.2s 3.2s 3.8s 2.6s 54321,0 days 00:00:33,3,20.4s 2.7s 2.5s 2.6s 2.4s 12345,0 days 00:00:34,18,17.7s 3.9s 3.0s 2.8s 2.7s 54321,0 days 00:00:33
3,1,4,33,Roman Rees,Rees,Roman,GER,3,14.5s 2.5s 2.6s 2.4s 2.4s 12345,0 days 00:00:28,17,15.6s 2.6s 2.5s 2.5s 2.4s 54321,0 days 00:00:28,4,16.8s 2.6s 2.7s 3.0s 3.9s 12045,0 days 00:00:32,19,16.0s 3.3s 2.6s 2.3s 2.2s 54321,0 days 00:00:30
4,1,5,11,Sebastian Samuelsson,Samuelsson,Sebastian,SWE,5,15.4s 3.5s 2.3s 2.4s 2.4s 12345,0 days 00:00:29,19,13.6s 4.1s 3.9s 2.7s 2.6s 04321,0 days 00:00:30,5,18.2s 3.2s 2.8s 2.8s 2.8s 12345,0 days 00:00:33,19,13.9s 2.9s 3.3s 3.5s 2.9s 54321,0 days 00:00:29


In [38]:
tmp = pd.read_csv('../../../tmp.txt', sep = '\t')
tmp

,Rank,Bib,Family Name,Given Name,Nation,Total Time,Course\nTime\nRank,back from\naverage\n(in %),back from\ntop10 average\n(in %),back from\ntop30 average\n(in %),back from\nmedian\n(in %),back from\ntop10 median\n(in %),back from\ntop30 median\n(in %),km/h,Course\nTime\nzScore
0,1,28,Ponsiluoma,Martin,SWE,44:38.4,2,-6.45,-0.84,-2.50,-6.84,-1.18,-2.62,26.12,-1.85
1,2,18,Hartweg,Niklas,SUI,46:12.7,21,-3.16,2.65,0.94,-3.55,2.30,0.80,25.23,-0.91
2,3,74,Zobel,David,GER,46:18.1,23,-2.97,2.85,1.13,-3.37,2.50,1.00,25.18,-0.85
3,4,33,Rees,Roman,GER,45:26.3,10,-4.78,0.93,-0.75,-5.17,0.59,-0.88,25.66,-1.37
4,5,11,Samuelsson,Sebastian,SWE,45:22.8,9,-4.90,0.80,-0.88,-5.29,0.46,-1.01,25.69,-1.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,93,69,Raenkel,Raido,EST,46:55.5,35,-1.66,4.23,2.49,-2.07,3.88,2.36,24.85,-0.48
93,94,76,Gunka,Jan,POL,48:43.3,68,2.10,8.23,6.42,1.68,7.86,6.28,23.93,0.60
94,95,65,Nakajima,Jin,JPN,49:23.3,82,3.50,9.71,7.88,3.07,9.33,7.73,23.61,1.00
95,96,64,Sirik,Sergey,KAZ,52:02.3,97,9.05,15.59,13.66,8.61,15.20,13.51,22.41,2.60


## Creating the Analysis table

In [56]:
create_analysis = "CREATE TABLE IF NOT EXISTS Analysis (\
    RaceID INT NOT NULL,\
    FinalRank SMALLINT NOT NULL,\
    Bib SMALLINT NOT NULL,\
    FullName varchar(30) NOT NULL,\
    FamilyName varchar(20) NOT NULL,\
    GivenName varchar(20) NOT NULL,\
    Nation char(3) NOT NULL,\
    TotalTime time,\
    CourseTimeRank SMALLINT,\
    BackFromAverage FLOAT,\
    BackFromTop10Average FLOAT,\
    BackFromTop30Average FLOAT,\
    BackFromMedian FLOAT,\
    BackFromTop10Median FLOAT,\
    BackFromTop30Median FLOAT,\
    kmh FLOAT,\
    CourseTimezScore FLOAT,\
    PRIMARY KEY (RaceID, Bib));"

In [7]:
tmp = pd.read_csv('../../../tmp.txt', sep = '\t')
tmp

,Rank,Bib,Family Name,Given Name,Nation,Total Time,Course\nTime\nRank,back from\naverage\n(in %),back from\ntop10 average\n(in %),back from\ntop30 average\n(in %),back from\nmedian\n(in %),back from\ntop10 median\n(in %),back from\ntop30 median\n(in %),km/h,Course\nTime\nzScore
0,1,28,Ponsiluoma,Martin,SWE,44:38.4,2,-6.45,-0.84,-2.50,-6.84,-1.18,-2.62,26.12,-1.85
1,2,18,Hartweg,Niklas,SUI,46:12.7,21,-3.16,2.65,0.94,-3.55,2.30,0.80,25.23,-0.91
2,3,74,Zobel,David,GER,46:18.1,23,-2.97,2.85,1.13,-3.37,2.50,1.00,25.18,-0.85
3,4,33,Rees,Roman,GER,45:26.3,10,-4.78,0.93,-0.75,-5.17,0.59,-0.88,25.66,-1.37
4,5,11,Samuelsson,Sebastian,SWE,45:22.8,9,-4.90,0.80,-0.88,-5.29,0.46,-1.01,25.69,-1.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,93,69,Raenkel,Raido,EST,46:55.5,35,-1.66,4.23,2.49,-2.07,3.88,2.36,24.85,-0.48
93,94,76,Gunka,Jan,POL,48:43.3,68,2.10,8.23,6.42,1.68,7.86,6.28,23.93,0.60
94,95,65,Nakajima,Jin,JPN,49:23.3,82,3.50,9.71,7.88,3.07,9.33,7.73,23.61,1.00
95,96,64,Sirik,Sergey,KAZ,52:02.3,97,9.05,15.59,13.66,8.61,15.20,13.51,22.41,2.60


In [10]:
dbc.execute("DESCRIBE Analysis;")
res_ = dbc.fetchall()
pd.DataFrame(res_)

,0,1,2,3,4,5
0,RaceID,int,NO,PRI,None,
1,FinalRank,smallint,NO,,None,
2,Bib,smallint,NO,PRI,None,
3,FullName,varchar(30),NO,,None,
4,FamilyName,varchar(20),NO,,None,
5,GivenName,varchar(20),NO,,None,
6,Nation,char(3),NO,,None,
7,TotalTime,time,YES,,None,
8,CourseTimeRank,smallint,YES,,None,
9,BackFromAverage,float,YES,,None,


In [16]:
dbc.execute('SELECT * FROM Analysis ORDER BY FinalRank LIMIT 5;')
res_ = dbc.fetchall()
pd.DataFrame(res_)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,1,28,Martin Ponsiluoma,Ponsiluoma,Martin,SWE,0 days 00:44:38,2,-6.45,-0.84,-2.50,-6.84,-1.18,-2.62,26.12,-1.85
1,1,2,18,Niklas Hartweg,Hartweg,Niklas,SUI,0 days 00:46:13,21,-3.16,2.65,0.94,-3.55,2.30,0.80,25.23,-0.91
2,1,3,74,David Zobel,Zobel,David,GER,0 days 00:46:18,23,-2.97,2.85,1.13,-3.37,2.50,1.00,25.18,-0.85
3,1,4,33,Roman Rees,Rees,Roman,GER,0 days 00:45:26,10,-4.78,0.93,-0.75,-5.17,0.59,-0.88,25.66,-1.37
4,1,5,11,Sebastian Samuelsson,Samuelsson,Sebastian,SWE,0 days 00:45:23,9,-4.90,0.80,-0.88,-5.29,0.46,-1.01,25.69,-1.41


In [17]:
tmp = pd.read_csv('../../../tmp.txt', sep = '\t')
tmp

,Rank,Bib,Family Name,Given Name,Nation,Day Time,Sector Time,Sector Rank,Net Time,Behind
0,1,28,Ponsiluoma,Martin,SWE,13:30:07,1:07.4,2,1:07.4,1.2
1,2,18,Hartweg,Niklas,SUI,13:25:09,1:10.2,20,1:10.2,4.0
2,3,74,Zobel,David,GER,13:53:11,1:11.6,39,1:11.6,5.4
3,4,33,Rees,Roman,GER,13:32:41,1:10.5,22,1:10.5,4.3
4,5,11,Samuelsson,Sebastian,SWE,13:21:38,1:07.8,5,1:07.8,1.6
...,...,...,...,...,...,...,...,...,...,...
92,93,69,Raenkel,Raido,EST,13:50:40,1:10.5,22,1:10.5,4.3
93,94,76,Gunka,Jan,POL,13:54:11,1:11.5,36,1:11.5,5.3
94,95,65,Nakajima,Jin,JPN,13:48:44,1:15.4,91,1:15.4,9.2
95,96,64,Sirik,Sergey,KAZ,13:48:12,1:13.2,67,1:13.2,7.0


## Creating the Split Times table

In [20]:
create_split_times = "CREATE TABLE IF NOT EXISTS SplitTimes (\
    RaceID INT NOT NULL,\
    FinalRank SMALLINT NOT NULL,\
    Bib SMALLINT NOT NULL,\
    km FLOAT,\
    FullName varchar(30) NOT NULL,\
    FamilyName varchar(20) NOT NULL,\
    GivenName varchar(20) NOT NULL,\
    Nation char(3) NOT NULL,\
    DayTime time,\
    SectorTime time,\
    SectorRank SMALLINT,\
    NetTime time,\
    Behind FLOAT,\
    PRIMARY KEY (RaceID, km, Bib));"

In [22]:
dbc.execute("DESCRIBE SplitTimes;")
res_ = dbc.fetchall()
pd.DataFrame(res_)

,0,1,2,3,4,5
0,RaceID,int,NO,PRI,None,
1,FinalRank,smallint,NO,,None,
2,Bib,smallint,NO,PRI,None,
3,km,float,NO,PRI,None,
4,FullName,varchar(30),NO,,None,
5,FamilyName,varchar(20),NO,,None,
6,GivenName,varchar(20),NO,,None,
7,Nation,char(3),NO,,None,
8,DayTime,time,YES,,None,
9,SectorTime,time,YES,,None,


In [24]:
dbc.execute('DESCRIBE SplitTimes;')
res_ = dbc.fetchall()
pd.DataFrame(res_)

,0,1,2,3,4,5
0,RaceID,int,NO,PRI,None,
1,FinalRank,smallint,NO,,None,
2,Bib,smallint,NO,PRI,None,
3,km,float,NO,PRI,None,
4,FullName,varchar(30),NO,,None,
5,FamilyName,varchar(20),NO,,None,
6,GivenName,varchar(20),NO,,None,
7,Nation,char(3),NO,,None,
8,DayTime,time,YES,,None,
9,SectorTime,time,YES,,None,


In [33]:
dbc.execute('SELECT * FROM SplitTimes ORDER BY FinalRank LIMIT 5;')
res_ = dbc.fetchall()
pd.DataFrame(res_)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,1,28,1.0,Martin Ponsiluoma,Ponsiluoma,Martin,SWE,0 days 13:30:07,0 days 00:01:07,2,0 days 00:01:07,0 days 00:00:01
1,1,2,18,1.0,Niklas Hartweg,Hartweg,Niklas,SUI,0 days 13:25:09,0 days 00:01:10,20,0 days 00:01:10,0 days 00:00:04
2,1,3,74,1.0,David Zobel,Zobel,David,GER,0 days 13:53:11,0 days 00:01:12,39,0 days 00:01:12,0 days 00:00:05
3,1,4,33,1.0,Roman Rees,Rees,Roman,GER,0 days 13:32:41,0 days 00:01:11,22,0 days 00:01:11,0 days 00:00:04
4,1,5,11,1.0,Sebastian Samuelsson,Samuelsson,Sebastian,SWE,0 days 13:21:38,0 days 00:01:08,5,0 days 00:01:08,0 days 00:00:02


In [89]:
dbc.close()

True

In [90]:
db.close()